# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [1]:
!cat publications.csv

pub_date,title,venue,excerpt,citation,url_slug,paper_url
2023-04-10,"Project Elements: A computational entity-component-system in a scene-graph pythonic framework, for a neural, geometric computer graphics curriculum","Eurographics 2023 - Education Papers ","We present the Elements project, a lightweight, open-source, computational science and computer graphics (CG) framework, tailored for educational needs, that offers, for the first time, the advantages of an Entity-Component-System (ECS) along with the rapid prototyping convenience of a Scenegraph-based pythonic framework. This novelty allows advances in the teaching of CG: from heterogeneous directed acyclic graphs and depth-first traversals, to animation, skinning, geometric algebra and shader-based components rendered via unique systems all the way to their representation as graph neural networks for 3D scientific visualization.","Papagiannakis, G., Kamarianakis, M., Protopsaltis, A., Angelis, D., Zikas, P., ""Project Elements: A

## Import pandas

We are using the very handy pandas library for dataframes.

In [2]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [3]:
publications = pd.read_csv("publications.csv", sep=",", header=0, encoding='ISO-8859-1')
publications


,pub_date,title,venue,excerpt,citation,url_slug,paper_url
0,2023-04-10,Project Elements: A computational entity-compo...,Eurographics 2023 - Education Papers,"We present the Elements project, a lightweight...","Papagiannakis, G., Kamarianakis, M., Protopsal...",paper-conference-38,http://papagiannakis.github.io/files/P38-2023.pdf
1,2023-03-20,MAGES 4.0: Accelerating the Worldï¿½s Transiti...,IEEE Comput. Graph. Appl.,"In this work, we propose MAGES 4.0, a novel so...","Zikas, P., Protopsaltis, A., Lydatakis, N., Ke...",paper-journal-39,https://ieeexplore.ieee.org/document/10038619
2,2014-04-20,glGA: an OpenGL Geometric Application framewo...,Eurographics 2014 - Education Papers,This paper presents the open-source glGA (Open...,"P19.\tPapagiannakis, G., Papanikolaou, P., Gre...",paper-conference-19,https://diglib.eg.org/handle/10.2312/eged.2014...
3,2013-11-15,Geometric algebra rotors for skinned character...,ACM SIGGRAPH Asia 2013,The main goal and contribution of this work is...,George Papagiannakis. 2013. Geometric algebra ...,paper-conference-18,https://dl.acm.org/doi/abs/10.1145/2542355.254...
4,2003-07-25,VHD++ Framework: Extendible Game Engine with R...,Computer Graphics International 2003,This paper presents the architecture of the VH...,"Ponder, M., Papagiannakis, G., Molet, T., Magn...",paper-conference-7,https://citeseerx.ist.psu.edu/document?repid=r...
5,2015-11-01,A fast and robust pipeline for populating mobi...,ACM SIGGRAPH Asia 2015,In this work we present a complete methodology...,"Papaefthymiou, M., Feng, A., Shapiro, A., Papa...",paper-conference-22,http://papagiannakis.github.io/files/P22-2015-...


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [4]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [5]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"
    
    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    #md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [6]:
!ls ../_publications/

2003-07-25-paper-conference-7.md       2015-11-01-paper-conference-22.md
2010-10-01-paper-title-number-2.md.old 2023-03-20-paper-journal-39.md
2013-11-15-paper-conference-18.md      2023-04-10-paper-conference-38.md
2014-04-20-paper-conference-19.md      2023-07-08-ComputationalMedicalXR.md


In [7]:
!cat ../_publications/2023-03-20-paper-journal-39.md 


---
title: "MAGES 4.0: Accelerating the Worldï¿½s Transition to VR Training and Democratizing the Authoring of the Medical Metaverse"
collection: publications
permalink: /publication/2023-03-20-paper-journal-39
excerpt: 'In this work, we propose MAGES 4.0, a novel software development kit to accelerate the creation of collaborative medical training applications in virtual/augmented reality (VR/AR). Our solution is essentially a low-code metaverse authoring platform for developers to rapidly prototype high-fidelity and high-complexity medical simulations. MAGES breaks the authoring boundaries across extended reality, since networked participants can also collaborate using different VR/AR as well as mobile and desktop devices, in the same metaverse world. With MAGES we propose an upgrade to the outdated 150-year-old masterï¿½apprentice medical training model. Our platform incorporates, in a nutshell, the following novelties: 1) 5G edge-cloud remote rendering and physics dissection layer,